In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt

In [ ]:
df = pd.concat(map(pd.read_csv, ['/kaggle/input/used-car-dataset-ford-and-mercedes/audi.csv', '/kaggle/input/used-car-dataset-ford-and-mercedes/bmw.csv','/kaggle/input/used-car-dataset-ford-and-mercedes/hyundi.csv', '/kaggle/input/used-car-dataset-ford-and-mercedes/merc.csv', '/kaggle/input/used-car-dataset-ford-and-mercedes/skoda.csv','/kaggle/input/used-car-dataset-ford-and-mercedes/toyota.csv','/kaggle/input/used-car-dataset-ford-and-mercedes/vauxhall.csv','/kaggle/input/used-car-dataset-ford-and-mercedes/ford.csv','/kaggle/input/used-car-dataset-ford-and-mercedes/vw.csv']))


In [ ]:
df

In [ ]:
df.info()

In [ ]:
sns.heatmap(df.isnull())

the missing value in the tax column was in the tax(£) column. I have to drop replace the null values in tax with the values in tax(£). 

In [ ]:
df.tax=df.tax.fillna(df[['tax(£)']].max(1))

In [ ]:
sns.heatmap(df.isnull())

We dont need the tax(£) again so we have to drop it 

In [ ]:
df.drop(['tax(£)'], axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr(),annot=True)

In [ ]:
fig = plt.figure(figsize=(18,6))
fig.add_subplot(1,2,1)
sns.countplot(df['transmission'])
fig.add_subplot(1,2,2)
sns.countplot(df['fuelType'])

In [ ]:
sns.catplot(x = 'year', y= 'price', data = df, kind='point', aspect=4);

In [ ]:
sns.relplot(x="price", y="transmission", 
            data=df);

In [ ]:
sns.relplot(x="year", y="price", 
            data=df);

In [ ]:
num_cols = df.select_dtypes(exclude=['object'])

fig = plt.figure(figsize=(20,8))

for col in range(len(num_cols.columns)):
    fig.add_subplot(2,4,col+1)
    sns.distplot(num_cols.iloc[:,col], hist=False, rug=True, kde_kws={'bw':0.1}, label='UV')
    plt.xlabel(num_cols.columns[col])

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(20,8))

for col in range(len(num_cols.columns)):
    fig.add_subplot(2,4,col+1)
    sns.scatterplot(x=num_cols.iloc[:,col], y=df['price'], label='MV')
    plt.xlabel(num_cols.columns[col])

plt.tight_layout()

##Feature Engineering The categorical variables must be converted into something numerical. either one hot encording from sklearn or using get dummies function from pandas

In [ ]:
cars

In [ ]:
X = cars.drop(['price'], axis=1)
y = cars['price']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from math import sqrt
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.svm import SVC
from sklearn.linear_model import Ridge,ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Lasso

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25) 

In [ ]:
log_clf=LinearRegression()
bye_clf=BayesianRidge()
rnd_clf = RandomForestRegressor()
rid_clf = Ridge(alpha=2,max_iter=1000,random_state=1)
ele_clf = ElasticNet()
gbr_clf=GradientBoostingRegressor()
lss_clf=Lasso()

In [ ]:
voting_clf = VotingRegressor([('lr', log_clf),('bye', bye_clf), ('rf', rnd_clf), ('rnd', rnd_clf), ('ele', ele_clf), ('gbr', gbr_clf), ('lss', lss_clf)])
voting_clf.fit(X_train, y_train)

In [ ]:
for clf in (log_clf, rnd_clf, rid_clf, bye_clf, voting_clf,ele_clf, gbr_clf,lss_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, 'r2_score', r2_score(y_test, y_pred))

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.20, random_state=7)

In [ ]:
voting_clf2 = VotingRegressor([('lr', log_clf),('bye', bye_clf), ('rf', rnd_clf), ('rnd', rnd_clf), ('ele', ele_clf), ('gbr', gbr_clf),('lss', lss_clf)])
voting_clf.fit(X_train, y_train)

In [ ]:
for clf in (log_clf, rnd_clf, rid_clf, bye_clf,voting_clf2,ele_clf, gbr_clf,lss_clf):
    clf.fit(X_train, y_train)
    y_pred2 = clf.predict(X_test)
    print(clf.__class__.__name__, 'r2_score', r2_score(y_test, y_pred2))

MinMaxScaler reduced the accuracy of RandomForestRegressor which was the best model to be used in this data it reduced the r2_score from  0.964 to 0.954 but it improved the Ridge r2_score to  0.8607171252232906 and also BayesianRidge r2_score 0.8635266838140631

In [ ]:
scaler2 = StandardScaler()
X_scaled2 = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled2, y, test_size=0.15, random_state=7)

In [ ]:
voting_clf3 = VotingRegressor([('lr', log_clf), ('rf', rnd_clf),('bye',bye_clf), ('rnd', rnd_clf), ('ele', ele_clf), ('gbr', gbr_clf),('lss', lss_clf)])
voting_clf.fit(X_train, y_train)

In [ ]:
for clf in (log_clf, rnd_clf, rid_clf, voting_clf2,ele_clf, gbr_clf,lss_clf):
    clf.fit(X_train, y_train)
    y_pred3 = clf.predict(X_test)
    print(clf.__class__.__name__, 'r2_score', r2_score(y_test, y_pred3))

# ***The result so far was without scaliing the data as it generated the best result with the RandomForestRegressor r2_score 0.964***